In [21]:
import json
import numpy as np
import pyperclip
from scipy.stats import ttest_ind_from_stats, ttest_rel, wilcoxon,ttest_ind,ttest_rel
import os
os.chdir(r"C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche")
from FIBI.data.analyse_cpp_results.clustering.extract_infos import parse_key, parse_metadata, DataCategory, corresp_impr_class
import pandas as pd
from scipy.stats import ranksums, mannwhitneyu
import plotly.express as px
import bs4
from itertools import product
from pathlib import Path
!pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager
!pip install selenium
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options
from scipy.stats import norm
import plotly.graph_objects as go
!pip install -U kaleido
!pip install pingouin
!pip install matplotlib
import pingouin as pg
import matplotlib.pyplot as plt
!pip install statsmodels
import statsmodels.stats.power as smp
from statsmodels.stats import weightstats as stests

# https://www.youtube.com/watch?v=CIbJSX-biu0 for ressources

In [2]:
from h5py import File
data_rd = []
with File("data/data_algo_same/table1_exp_clust_1000samples.hdf5", "r") as f:
    for k,v in f["metadata"].items():
        dico_key = parse_key(k)
        v = np.copy(v)
        dico_value = {"key_dataset":k,**parse_metadata(v)}
        data_rd.append(dico_value)
df_rd = pd.DataFrame(data_rd)
npoints = list(df_rd['NUM_POINTS'].unique())
nclust = list(df_rd['NUM_CLUST'].unique())
Limpr = list(df_rd['IMPR_CLASS'].unique())
for n,nclus,impr in product(npoints,nclust,Limpr):
    if nclus>=n:
        continue
    d=df_rd.query(f"NUM_POINTS == {n} & NUM_CLUST == {nclus} & IMPR_CLASS == '{impr}'")
    assert len(d) == 1000, f"Problem for NUM_POINTS == {n} & NUM_CLUST == {nclus} & IMPR_CLASS == '{impr}': found: {len(d)}"
data_impr = []
with File("data/data_algo_same/table1_exp_clust_impr_1000samples.hdf5", "r") as f:
    for k,v in f["metadata"].items():
        dico_key = parse_key(k)
        v = np.copy(v)
        dico_value = {"key_dataset":k,**parse_metadata(v)}
        data_impr.append(dico_value)
df_impr = pd.DataFrame(data_impr)
for n,nclus,impr in product(npoints,nclust,Limpr):
    if nclus>=n:
        continue
    d=df_impr.query(f"NUM_POINTS == {n} & NUM_CLUST == {nclus} & IMPR_CLASS == '{impr}'")
    assert (len(d) == 1000), f"Problem for NUM_POINTS == {n} & NUM_CLUST == {nclus} & IMPR_CLASS == '{impr}': found: {len(d)}"

df_impr['INIT'] = 'IMPR'
df_rd['INIT'] = 'RAND'

df = pd.concat([df_rd,df_impr])


### Search for seed and visualize initial cluster

In [66]:
df_21 = df.query(f"NUM_POINTS == 50 & NUM_CLUST == 16 & INIT == 'RAND' & IMPR_CLASS == 'FICL'").copy()
df_21.loc[:,"ratio"] = df_21.loc[:,'final_cost']/df_21.loc[:,'init_cost']
df_21.sort_values(by=['SEED_POINTS','SEED_ASSIGN'],inplace=True)
df_21 = df_21[['ratio','key_dataset','SEED_GLOB']]
print(df_21.iloc[968])
# display(.head(20))

ratio                             0.075505
key_dataset    55968,968,50,0,968,1,16,0,1
SEED_GLOB                            55968
Name: 317077, dtype: object


#### Visualize

In [72]:
a=np.array([[5,5],[2,2],[4,4]])
a[a==5]

array([5, 5])

In [ ]:
path = Path(r'C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche\cpp_generation\clustering\x64\Release\dataset.hdf5')
with File(path) as f:
    points = np.copy(f['points_coords']['54968,968,50,0,968,0,16,0,1']).reshape((-1,2))
    init_bi = np.copy(f['final_assignements']['54968,968,50,0,968,0,16,0,1'])
    init_fi = np.copy(f['final_assignements']['55968,968,50,0,968,1,16,0,1'])
Ldf = []
for [x,y],bi,fi in zip(points,init_bi,init_fi):
    Ldf.append({'x':x,'y':y,'fi_clust':fi,'bi_clust':bi})
with open('points.txt','w') as f:
    f.write('\n'.join(['\t\t'.join(map(str,p)) for p in points]))
with open('fi.txt','w') as f:
    f.write('\n'.join([str(p) for p in init_fi]))
with open('bi.txt','w') as f:
    f.write('\n'.join([str(p) for p in init_bi]))
df_22 = pd.DataFrame(Ldf)
for algo in ['fi','bi']:
    px.scatter(df_22,x="x",y="y", opacity=0,text=algo+"_clust",width=500,height=500,title=algo).show()
centroids = {k for k in init_bi}
cost = 0
for centroid in centroids:
    centr_points = points[init_bi==centroid]
    centroid = np.sum(centr_points,axis=0)/centr_points.shape[0]
    cost += ((centr_points-centroid)**2).sum()
print("cost BI:",cost)
cost = 0
for centroid in centroids:
    centr_points = points[init_fi==centroid]
    centroid = np.sum(centr_points,axis=0)/centr_points.shape[0]
    cost += ((centr_points-centroid)**2).sum()
print("cost FI:",cost)

## Observation definition of "best algorithm"

## Recompute the averages as in FIvsBI

In [3]:

page = bs4.BeautifulSoup("""<html><head><link rel="stylesheet" href="style5.css"></head><body></body></html>""")
for init in df['INIT'].unique():
    df_vis = df.query(f'INIT == "{init}"')
    algorithms = ['BICL','FICL'][::-1]
    make_key1 = lambda npts,algo,nclust:f"{npts}_{algo}_{nclust}"
    key_to_dict1 = lambda x:{"npts":x.split('_')[0],"algo":x.split('_')[1],"nclust":x.split('_')[2]}
    dico = {}
    for npts,nclust,algo in product(df['NUM_POINTS'].unique(),df['NUM_CLUST'].unique(),algorithms):
        if npts <= nclust:
            continue
        df_vis2 = df_vis.query(f"NUM_POINTS == {npts} & NUM_CLUST == {nclust}")
        df_vis2 = df_vis2.query(f"IMPR_CLASS == '{algo}'")
        df_vis2.sort_values(by=['SEED_POINTS','SEED_ASSIGN'],inplace=True)
        dico[make_key1(npts,algo,nclust)] = {"meanFinalCost":np.mean(df_vis2['final_cost']),"meanFinalCostInitCost":np.mean(df_vis2['final_cost']/df_vis2['init_cost']), "data":np.array(df_vis2['final_cost']),'init':np.array(df_vis2['init_cost'])}
    Ldf = []
    for npts in sorted(df['NUM_POINTS'].unique()):
        d = {}
        d[('','',"npoints")] = npts
        for nclust in sorted(df['NUM_CLUST'].unique()):
            if npts <= nclust:
                # for algo in algorithms:
                #     d[(str(nclust)+" clusters",'Objective',algo)] = None
                #     d[(str(nclust)+" clusters",'ratio', algo)] = None
                d[(str(nclust)+" clusters",'impr','')] =  None
                d[(str(nclust)+" clusters",'norm. cost diff', '')] = None
            else:
                # for algo in algorithms:
                #     d[(str(nclust)+" clusters",'Objective',algo)] = dico[make_key1(npts,algo,nclust)]['meanFinalCost']
                #     d[(str(nclust)+" clusters",'ratio', algo)] = np.mean((dico[make_key1(npts,algo,nclust)]['data']/dico[make_key1(npts,algo,nclust)]['init']))
                bi_data = dico[make_key1(npts,'BICL',nclust)]['data']
                fi_data = dico[make_key1(npts,'FICL',nclust)]['data']
                init_cost = dico[make_key1(npts,'FICL',nclust)]['init']
                d[(str(nclust)+" clusters",'impr', '')] = np.mean((bi_data-fi_data)/fi_data*100)
                d[(str(nclust)+" clusters",'norm. cost diff', '')] = np.mean((bi_data-fi_data)/init_cost)
        Ldf.append(d)
    dico_val = {}
    for l in Ldf:
        for k,v in l.items():
            if '_'.join(k) not in dico_val:
                dico_val['_'.join(k)] = []
            dico_val['_'.join(k)].append(v)
    new_dico_val = {}
    for k,v in dico_val.items():
        new_dico_val[tuple(k.split('_'))] = v
    table1 = pd.DataFrame(new_dico_val).set_index(keys=[('','',"npoints")])
    table1.index.names = ['N points']
    table1 = table1.applymap(lambda x: f"{x:0.3f}"if pd.notnull(x) else '-').to_html()
    page.html.body.append(bs4.BeautifulSoup(f"<h2>Initialization {init}</h2>"))
    table1 = bs4.BeautifulSoup(table1)
    for i in range(len(df['NUM_CLUST'].unique())+1):
        for th in table1.select(f"table > thead > tr > th:nth-child({3+2*i})"):
            th['title']='(FinalCost(BICL)-FinalCost(FICL))/FinalCost(FICL)'
        for td in table1.select(f"table > tbody > tr > td:nth-child({6+6*i})"):
            if td.text != '-':
                if float(td.text) > 0:
                    td['class'] = 'positive'
                elif float(td.text) == 0:
                    td['class'] = ''
                else:
                    td['class'] = 'negative'
        for td in table1.select(f"table > tbody > tr > td:nth-child({7+6*i})"):
            if td.text != '-':
                if float(td.text) > 0:
                    td['class'] = 'positive'
                elif float(td.text) == 0:
                    td['class'] = ''
                else:
                    td['class'] = 'negative'
    page.html.body.append(table1)
folder  = Path('FIBI/data/detailed_analysis/graphs/')
name = f"averages.html"
folder.mkdir(parents=True, exist_ok=True)
with open(str(folder / name),"w") as f:
    f.write(str(page))

## Initial cost variations

In [4]:
df_20 = df.copy()
Lpts = df_20['NUM_POINTS'].unique()
Lclust = df_20['NUM_CLUST'].unique()
Linit = df_20['INIT'].unique()
Ldf = []
possibilities = filter(lambda x:x[0] > x[1], product(Lpts,Lclust,Linit))
for npts,nclust,init in possibilities:
    df_tmp = df_20.query(f"NUM_CLUST == {nclust} & NUM_POINTS == {npts} & INIT == '{init}' & IMPR_CLASS == 'FICL'")
    inits = np.array(df_tmp['init_cost'])
    mean = np.mean(inits)
    std = np.std(inits)
    normalized_std = std/mean
    Ldf.append({'mean':mean,'std':std,'normalized_std':normalized_std})
df_vis = pd.DataFrame(Ldf)
fig = px.strip(df_vis,y='normalized_std',stripmode='overlay')
fig.add_trace(go.Box(y=df_vis["normalized_std"], boxpoints=False,name=''))
fig.update_layout(showlegend=False)
fig.update_traces(marker=dict(color='red'))
fig.update_layout(
    yaxis_title=r'σ normalized'
)
fig.show()

## Statistics per possibilities

In [3]:
LnPts = df['NUM_POINTS'].unique()
LnPts.sort()
Lassign = ['RAND','IMPR']
Lalgo = ['BICL','FICL']
Lnclust = df['NUM_CLUST'].unique()
Lnsamples = [100,1000]

In [27]:
# Taken from https://github.com/scipy/scipy/blob/v1.2.1/scipy/stats/morestats.py#L2709-L2806 (on advice of https://github.com/scipy/scipy/issues/2625)
from numpy import (asarray, compress) # requiered for local modification
from scipy.stats import (find_repeats, distributions,rankdata)
from math import sqrt

def wilcoxon_modified(x, y=None, zero_method="wilcox", correction=False):
    if zero_method not in ["wilcox", "pratt", "zsplit"]:
        raise ValueError("Zero method should be either 'wilcox' "
                         "or 'pratt' or 'zsplit'")

    if y is None:
        d = asarray(x)
    else:
        x, y = map(asarray, (x, y))
        if len(x) != len(y):
            raise ValueError('Unequal N in wilcoxon.  Aborting.')
        d = x - y

    if zero_method == "wilcox":
        # Keep all non-zero differences
        d = compress(np.not_equal(d, 0), d, axis=-1)

    count = len(d)
    if count < 10:
        warnings.warn("Warning: sample size too small for normal approximation.")

    r = rankdata(abs(d))
    r_plus = np.sum((d > 0) * r, axis=0)
    r_minus = np.sum((d < 0) * r, axis=0)

    if zero_method == "zsplit":
        r_zero = np.sum((d == 0) * r, axis=0)
        r_plus += r_zero / 2.
        r_minus += r_zero / 2.

    T = min(r_plus, r_minus)
    mn = count * (count + 1.) * 0.25 # n(n+1)/4 => if no difference btwn algo, T+ ~ T-: here n(n+1)/4 = half ranks in T+, half in T-
    se = count * (count + 1.) * (2. * count + 1.) # std from the mid separation T+ ~ T-

    if zero_method == "pratt":
        r = r[d != 0]

    replist, repnum = find_repeats(r)
    if repnum.size != 0:
        # Correction for repeated elements.
        se -= 0.5 * (repnum * (repnum * repnum - 1)).sum()

    se = sqrt(se / 24) # why 24 = 6 x 4 <- 6 ok (n(n+1)(2n+1)/6) but 1/4 ? -> var^2/2=sigma/2
    correction = 0.5 * int(bool(correction)) * np.sign(T - mn) # = 0 here
    z = (T - mn - correction) / se # compare value occured with theorical value if no difference, normalized by std
    prob = 2. * distributions.norm.sf(abs(z)) # sf = survival function of the norm scipy package S([|z|]) = Pr(x>|z|)
    # Here we take 2 times the probability that x is greater than |z|, normalized deviation from theorical equal repartition of ranks
    # Multiply by 2 because 2 sides p values (_|_/|-|\_|)
    # T-mn represents the biggest gap between theorical equal repartition of ranks between R+ and R- and observed repartition of ranks
    # se normalize this value by the sum of ranks

    return {"T":T,"pvalue":prob,"Z":z,"sigma":se,"mu":mn,"S_plus":r_plus,"S_minus":r_minus,"n":count,"rep":len(repnum),"correction":correction, "effect_size":z/count**0.5}
def ztest(a,b):
    diff = a-b
    n = diff.shape[0]
    d = (diff).astype(np.float64)
    v = np.var(d, 0, ddof=1)
    dm = np.mean(d, 0)
    denom = np.sqrt(v / n)
    t = (dm/denom)
    df=n-1
    prob = distributions.norm.sf(np.abs(t)) * 2  # use np.abs to get upper tail
    cohens = dm/v**0.5
    return {"z":t,"pvalue":prob,"sigmaNorm":denom,"mean":dm,'effect_size':cohens}
def sign_test(diff):
    # Source: https://openpress.usask.ca/introtoappliedstatsforpsych/chapter/16-2-median-sign-test/
    n = len(diff != 0)
    r_plus = np.sum((diff > 0), axis=0)
    r_minus = np.sum((diff < 0), axis=0)
    X = min(r_plus,r_minus)
    if X<n/2:
        Z = ((X+0.5)-n/2)/(n**0.5/2)
    else:
        Z = ((X-0.5)-n/2)/(n**0.5/2)
    prob = 2. * distributions.norm.sf(abs(Z))
    return {"z":Z,"pvalue":prob,"r_plus":r_plus,"r_minus":r_minus,'n':n,'effect_size':X/n-0.5}
def rank_biserial_corr(stat,n):
    T = stat
    R1 = abs(stat)
    R2 = abs(n*(n+1)/2-R1)
    return 4*abs(T-(R1+R2)/2)/(n*(n+1))
# Source effect size https://www.utstat.toronto.edu/~brunner/oldclass/378f16/readings/CohenPower.pdf page 164
# ttest source : https://github.com/scipy/scipy/blob/c3fa90dcfcaef71658744c73578e9e7d915c81e9/scipy/stats/stats.py#L4183


### Compute stats per possibility

In [5]:

%%capture 

to_key = lambda npts,nsamples,nclust,init: f"{npts}_{nsamples}_{nclust}_{init}"
key_to_dict = lambda x: {'npts':x.split('_')[0],'nsamples':x.split('_')[1],'nclust':x.split('_')[2], 'INIT':x.split('_')[3]}

service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get('https://www.statskingdom.com/paired-t-test-calculator.html')
fi_textarea = driver.find_element("id","c0")
bi_textarea = driver.find_element("id","c1")
bi_textarea.clear()
submit_btn = driver.find_element("id","manual_")
import time
time.sleep(1)
updatemenusYAxis = list([
    dict(active=1,
         buttons=list([
            dict(label='Log Scale',
                 method='update',
                 args=[{'visible': [True, True]},
                       {'title': 'Log scale',
                        'yaxis': {'type': 'log'}}]),
            dict(label='Linear Scale',
                 method='update',
                 args=[{'visible': [True, True]},
                       {'title': 'Linear scale',
                        'yaxis': {'type': 'linear'}}])
            ]),
        )
    ])
res_ttest = {}
res_wilcoxon = {}
res_sign_test = {}
res_best = {}
res_variance = {}
Lfig_names = ['box_algos','histo_algos','box_diff','histo_diff','qqplot','symmetric plot']
Lfig_names_log = [e for e in Lfig_names if e not in ['qqplot','symmetric plot']]
# Data preprocessing
for nPts,INIT,N_CLUST,nsamples in product(LnPts,Lassign,Lnclust,Lnsamples):
    if nPts <= N_CLUST:
        continue
    df_1 = df.query(f"INIT == '{INIT}' & NUM_CLUST=={N_CLUST} & NUM_POINTS == {nPts}").copy()
    # Graph utilitary elements
    dico_fig = {}
    for name in Lfig_names:
        if name not in Lfig_names_log:
            dico_fig[name] = go.Figure(layout=dict(updatemenus=updatemenusYAxis, title='Linear scale'))
        else:
            dico_fig[name] = go.Figure()
    dico = {}
    for algo,field,color in zip(Lalgo,["c0","c1"],['red','blue']):
        driver.execute_script(f"document.querySelector('#{field}').value = ''")
        # Preprocess data
        df_3 = df_1.query(f"IMPR_CLASS=='{algo}'")
        df_3.sort_values(by=['SEED_POINTS','SEED_ASSIGN'],inplace=True)
        final_cost = np.array(df_3['final_cost'])[:nsamples]
        init_cost = np.array(df_3['init_cost'])[:nsamples]
        dico[algo] = (final_cost/init_cost)
        # Fill html form 
        data = ",".join([str(e) for e in dico[algo]])
        driver.execute_script(f"document.querySelector('#{field}').value = '{data}'")
    for algo,color in zip(Lalgo,['red','blue']):
        # Create graphs
        dico_fig['box_algos'].add_trace(go.Box(y=dico[algo],name=f"{nPts} points, algo {algo}",boxpoints="all"))
        min_algos = min(np.min(dico['FICL']),np.min(dico['BICL']))
        max_algos = max(np.max(dico['FICL']),np.max(dico['BICL']))
        x=np.linspace(min_algos,max_algos,num=1000)
        mu=np.mean(dico[algo])
        sigma=np.std(dico[algo])
        hist,bins_edges = np.histogram(dico[algo],bins=50,
                                       range=(min_algos,max_algos)
                                       )
        area = 0
        for i in range(len(hist)):
            area += (bins_edges[i+1]-bins_edges[i])*hist[i]
        means = [(bins_edges[i]+bins_edges[i+1])/2 for i in range(len(hist))]
        dico_fig['histo_algos'].add_trace(go.Bar(x=means,y=hist,name=f"{nPts} points, algo {algo}",marker=dict(color = color)))
        dico_fig['histo_algos'].add_trace(go.Scatter(x=x,y=norm.pdf(x, mu, sigma)*area,marker=dict(color = color),name=f"{nPts} points, algo {algo}, normal distrib."))
        dico_fig['histo_algos'].update_layout(barmode='group', bargap=0.0,bargroupgap=0.0)
    time.sleep(0.1)
    # Compute statistics
    ## On the web for ttest
    driver.execute_script("calculate_paired_t()");
    pvalue = driver.execute_script("return document.querySelector('#report tr td:nth-child(2) div').textContent")
    cohens_d = driver.execute_script("return document.querySelector('#steps1 table:last-child tr:first-child td:last-child').innerText")
    res_ttest[to_key(nPts,nsamples,N_CLUST,INIT)] = {
        "pvalue":pvalue,
        "effect_size":cohens_d.split('=')[1].strip(),
        'tvalue':compute_t(dico[Lalgo[0]]-dico[Lalgo[1]]),
        'mean':np.mean(dico[Lalgo[0]]-dico[Lalgo[1]]),
        'sigmaNorm':(np.var(dico[Lalgo[0]]-dico[Lalgo[1]],ddof=1)/dico[Lalgo[0]].shape[0])**0.5,
        }
    ## Locally for wilcoxon
    r_wilcoxon = wilcoxon_modified(dico['BICL'],dico['FICL'])
    res_wilcoxon[to_key(nPts,nsamples,N_CLUST,INIT)] = r_wilcoxon
    ## Locally for sign test
    res_sign_test[to_key(nPts,nsamples,N_CLUST,INIT)] = sign_test(dico[Lalgo[0]]-dico[Lalgo[1]])
    ## Same variance check
    res_variance[to_key(nPts,nsamples,N_CLUST,INIT)] = {algo:np.var(dico[algo]) for algo in dico}
    
    # Create graphs for difference
    ## Box plot
    dico_fig['box_diff'].add_trace(go.Box(y=dico[Lalgo[0]]-dico[Lalgo[1]],name=f"{nPts} points, difference",boxpoints="all"))
    best_in_avg = 'BICL' if np.mean(dico['FICL']-dico['BICL']) > 0 else 'FICL'
    res_best[to_key(nPts,nsamples,N_CLUST,INIT)] = best_in_avg
    ## Histogram
    dico_fig['hist_diff'].add_trace(go.Histogram(x=dico[Lalgo[0]]-dico[Lalgo[1]],name=f"{nPts} points, difference"))
    ### Overlay histograms
    dico_fig['hist_diff'].update_layout(barmode='overlay')
    dico_fig['hist_diff'].update_traces(opacity=0.75)
    ## QQPlot
    qqplot = pg.qqplot(dico[Lalgo[0]]-dico[Lalgo[1]], sparams=(0, 1))
    names_mapping = {0:"Data",1:"Trend of the data",4:"Bound interval 95%",5:"Bound interval 95%"}
    for i,l in enumerate(qqplot._children):
        if i in [2,3]:
            continue
        dico_fig['qqplot'].add_trace(go.Scatter(
            x=l._x,
            y=l._y,
            name=names_mapping[i],
            mode="markers" if i==0 else None, 
            marker=None if i != 0 else dict(size=5,line=dict(width=2))
            )
                              )
    dico_fig['qqplot'].update_layout(xaxis_title='Theorical quantiles',yaxis_title='Samples quantiles')
    diff = (dico[Lalgo[0]]-dico[Lalgo[1]])
    median = np.median(diff)
    sym_data = np.sort(diff)
    m,M = np.min(sym_data),np.max(sym_data)
    identity = np.linspace(m,M,1000)
    dico_fig['symmetric plot'].add_trace(go.Scatter(x=sym_data[::-1]-median,y=median-sym_data,name='Data',mode='markers'))
    dico_fig['symmetric plot'].add_trace(go.Scatter(x=identity,y=identity,name='Symetric plot',mode='lines'))
    
    
    # Create webpages to show results
    webpage = bs4.BeautifulSoup("""<html><head><link rel="stylesheet" href="style3.css"></head><body></body></html>""")
    webpage.html.body.append(bs4.BeautifulSoup(f"<h2>{nPts} points limited to {len(dico['FICL'])}, {INIT} initializaion, {N_CLUST} clusters, {nsamples} samples</h2>"))
    webpage.html.body.append(bs4.BeautifulSoup(f"<p>Best in average {best_in_avg}</p>"))
    webpage.html.body.append(bs4.BeautifulSoup(f"<h3>Statistics</h3>"))
    table = bs4.BeautifulSoup(f"<table></table>")
    table.table.append(bs4.BeautifulSoup(f"<tr><th>Test/Values</th><th>PValue</th><th>Effect size</th></tr>"))
    table.table.append(bs4.BeautifulSoup(f"<tr><th>TTest</th><td>{float(res_ttest[to_key(nPts,nsamples,N_CLUST,INIT)]['pvalue']):.5e}</td><td>{float(res_ttest[to_key(nPts,nsamples,N_CLUST,INIT)]['effect_size']):.5e}</td></tr>"))
    table.table.append(bs4.BeautifulSoup(f"<tr><th>Wilcoxon</th><td>{float(res_wilcoxon[to_key(nPts,nsamples,N_CLUST,INIT)]['pvalue']):.5e}</td><td>{float(res_wilcoxon[to_key(nPts,nsamples,N_CLUST,INIT)]['effect_size']):.5e}</td></tr>"))
    table.table.append(bs4.BeautifulSoup(f"<tr><th>Sign test</th><td>{float(res_sign_test[to_key(nPts,nsamples,N_CLUST,INIT)]['pvalue']):.5e}</td>{float(res_sign_test[to_key(nPts,nsamples,N_CLUST,INIT)]['effect_size']):.5e}<td></td></tr>"))
    webpage.html.body.append(table)
    tvalue = res_ttest[to_key(nPts,nsamples,N_CLUST,INIT)]['tvalue']
    mu = res_ttest[to_key(nPts,nsamples,N_CLUST,INIT)]['mean']
    std = res_ttest[to_key(nPts,nsamples,N_CLUST,INIT)]['sigmaNorm']
    webpage.html.body.append(bs4.BeautifulSoup(f'<p>t value:{tvalue},sigma_norm:{std}, mu:{mu}</p>'))
    webpage.html.body.append(bs4.BeautifulSoup(f'<p>Wilcoxon:{",".join([k+":"+str(e) for k,e in res_wilcoxon[to_key(nPts,nsamples,N_CLUST,INIT)].items()])}</p>'))
    webpage.html.body.append(bs4.BeautifulSoup(f'<p>Sign test:{",".join([k+":"+str(e) for k,e in res_sign_test[to_key(nPts,nsamples,N_CLUST,INIT)].items()])}</p>'))
    table = bs4.BeautifulSoup(f"<table><tr><th></th><th>FICL</th><th>BICL</th><th>FICL-BICL</th></tr></table>")
    stats = {}
    for name,data in [('FICL',dico['FICL']),('BICL',dico['BICL']),('FICL-BICL',dico['FICL']-dico['BICL'])]:
        stats[name] = pd.DataFrame(data,columns=['value']).describe().reset_index()
    for name,values in zip(stats['FICL']['index'],zip(*[stats[algo]['value'] for algo in stats])):
        table.table.append(bs4.BeautifulSoup(f"<tr><th>{name}</th>{''.join([f'<td>{value}</td>' for value in values])}</tr>"))
    webpage.html.body.append(table)
    webpage.html.body.append(bs4.BeautifulSoup(f"<h3>Graphs</h3>"))
    for plot,text,id in [
        ["Box plot algorithms","boxAlgo"],
        ["Histograms of algorithms","histoAlgos"],
        ["Box plot of difference "+Lalgo[0]+'-'+Lalgo[1],'boxDiff'],
        ["Histogram of difference "+Lalgo[0]+'-'+Lalgo[1],"histoDiff"],
        ["QQPlot of "+Lalgo[0]+'-'+Lalgo[1],"qqplot"],
        ["Symmetry plot of "+Lalgo[0]+'-'+Lalgo[1],"symmetryPlot"]
        ]:
        dico_fig[id].update_layout(title=text)
        webpage.html.body.append(bs4.BeautifulSoup(f"<h2 id='{id}'>{text}</h2>"))
        webpage.html.body.append(bs4.BeautifulSoup(dico_fig[id].to_html(full_html=False, include_plotlyjs='cdn')))
    div = bs4.BeautifulSoup("<div id='copyData'><button onclick='copyFI()'>Copy FI data</button><button onclick='copyBI()'>Copy BI data</button></div>")
    data = ",".join([str(e) for e in dico['FICL']])
    div.div.append(bs4.BeautifulSoup('<script>function copyFI(){navigator.clipboard.writeText("'+data+'")}</script>'))
    data = ",".join([str(e) for e in dico['BICL']])
    div.div.append(bs4.BeautifulSoup('<script>function copyBI(){navigator.clipboard.writeText("'+data+'")}</script>'))
    webpage.html.body.append(div)
    
    folder  = Path('FIBI/data/detailed_analysis/graphs/')
    name = f"{nPts}_points_init_{INIT}_{N_CLUST}_clusters_{nsamples}_samples.html"
    folder.mkdir(parents=True, exist_ok=True)
    with open(str(folder / name),"w") as f:
        f.write(str(webpage))
driver.quit()

KeyboardInterrupt: 

### Compute global table pvalues

In [53]:
# LnPts = df['NUM_POINTS'].unique()
# LnPts.sort()
# Lassign = ['RAND','IMPR']
# Lalgo = ['BICL','FICL']
# Lnclust = df['NUM_CLUST'].unique()
# Lnsamples = [100,1000]
LnPts = df['NUM_POINTS'].unique()
LnPts.sort()
Lassign = ['RAND','IMPR']
Lalgo = ['BICL','FICL']
Lnclust = df['NUM_CLUST'].unique()
Lnclust.sort()
Lnsamples = [1000]

In [55]:

page = bs4.BeautifulSoup("""<html><head><link rel="stylesheet" href="style4.css"></head><body></body></html>""")
page.html.body.append(bs4.BeautifulSoup("<p>Null hypothesis: the two algorithms have identical average values (here ratio finalCost/initCost)</p>"))
page.html.body.append(bs4.BeautifulSoup("<p>P-Value < 0.05 => can reject the null hypothesis"))
dico_names_effect_size = {
    "TTest":"<a href='https://www.spss-tutorials.com/cohens-d/' target='_blank'>Cohen's d",
    "Wilcoxon":"<a href='https://www.researchgate.net/publication/303919832_The_need_to_report_effect_size_estimates_revisited_An_overview_of_some_recommended_measures_of_effect_size' target='_blank'>Z-Score</a>",
    "Sign test":"<a href='https://www.utstat.toronto.edu/~brunner/oldclass/378f16/readings/CohenPower.pdf'>Sign test</a>"
    }

def cohens_d_color(x):
    a=abs(x)
    if a < 0.5:
        return 'cohensSmall small'
    elif 0.5 <= a < 0.8:
        return 'cohensMedium medium'
    else:
        return 'cohensBig big'
def ztest(x):
    a=abs(x)
    if a < 0.3:
        return 'zSmall small'
    elif 0.3 <= a < 0.5:
        return 'zMedium medium'
    else:
        return 'zBig big'
def sign_test_color(x):
    a=abs(x)
    if a < 0.15:
        return 'signTestSmall small'
    elif 0.15 <= a < 0.25:
        return 'signTestMedium medium'
    else:
        return 'signTestBig big'
    
pvalue_color = lambda x:'pvalueSmall' if float(x)<5e-2 else 'pvalueBig'
colors = {
    'TTest':{'pvalue':pvalue_color,'effect_size':cohens_d_color},
    'Wilcoxon':{'pvalue': pvalue_color, 'effect_size':ztest},
    'Sign test':{'pvalue': pvalue_color, 'effect_size':sign_test_color}}
for N_CLUST,nsamples in product(sorted(Lnclust),sorted(Lnsamples)):
    title = bs4.BeautifulSoup(f"""<h2>{N_CLUST} clusters, {nsamples} samples</h2>""")
    page.html.body.append(title)
    div = bs4.BeautifulSoup('<div class="legend"></div>')
    size = 10
    divpvalue = bs4.BeautifulSoup('<div>P-Value:</div>')
    for id_var,text in [['small','<5e-2 = difference btw algos'],['big','>=5e-2 = no difference btw algos']]:
        div1 = bs4.BeautifulSoup(f'<div><svg width="{size}px" height="{size}px"><rect fill="var(--{id_var})" width="{size}px" height="{size}px"></rect></svg>{text}</div>')
        divpvalue.div.append(div1)
    div.div.append(divpvalue) 
    div.div.append(bs4.BeautifulSoup('<span></span>')) 
    diveffectsize = bs4.BeautifulSoup('<div class="legend">Effect-size:</div>')
    for id_var,text in [
        ['small','small (cohen s d:0.2 ; z-test: 0.1 ; sign-test: 0.05)'],
        ['medium','medium (cohen s d:0.5 ; z-test: 0.3 ; sign-test: 0.15)'],
        ['big','big (cohen s d:0.8 ; z-test: 0.5 ; sign-test: 0.25)']
        ]:
        div1 = bs4.BeautifulSoup(f'<div><svg width="{size}px" height="{size}px"><rect fill="var(--{id_var})" width="{size}px" height="{size}px"></rect></svg>{text}</div>')
        diveffectsize.div.append(div1)
        
    div.div.append(diveffectsize) 
    page.html.body.append(div) 
    legend = bs4.BeautifulSoup(f"""<div><svg><rect fill></rect></svg></div>""")
    page.html.body.append(title)
    table = bs4.BeautifulSoup(f"<table></table>")
    first_line = bs4.BeautifulSoup(f'<tr><th colspan="3"></th><th colspan="{len(LnPts)}">Number of points</th></tr>')
    second_line = bs4.BeautifulSoup(f'<tr><th colspan="3"></th>{"".join([f"<th>{e}</th>" for e in LnPts])}</tr>')
    table.table.append(first_line)
    table.table.append(second_line)
    for init in Lassign:
        init_done = False
        for i,(testName,testDico) in enumerate([['TTest',res_ttest],['Sign test',res_sign_test]]):# ,['Wilcoxon',res_wilcoxon]
            test_done = False
            for j,stat in enumerate(['pvalue','effect_size']):
                test_name_done = False
                tr = bs4.BeautifulSoup(f'<tr></tr>')
                if i == 0 and not init_done:
                    tr.tr.append(bs4.BeautifulSoup(f'<th rowspan={4}><a href="./pvalue_evolution.html#{N_CLUST}_{init}" target="_blank">{init}</a></th>'))
                    init_done = True
                if j == 0 and not test_done:
                    n = testName
                    if i == 0:
                        n += '<br>(online)'
                    tr.tr.append(bs4.BeautifulSoup(f'<th rowspan={2}>{n}</th>'))
                    test_done = True
                name = 'P-Value' if stat == "pvalue" else dico_names_effect_size[testName]
                tr.tr.append(bs4.BeautifulSoup(f'<th rowspan={1}>{name}</th>'))
                for k,nPts in enumerate(sorted(LnPts)):
                    if nPts<=N_CLUST:
                        tr.tr.append(bs4.BeautifulSoup('<td></td>'))
                    else:
                        val = float(testDico[to_key(nPts,nsamples,N_CLUST,init)][stat])
                        url = f"{nPts}_points_init_{init}_{N_CLUST}_clusters_{nsamples}_samples.html"
                        tr.tr.append(bs4.BeautifulSoup(f'<td class="{colors[testName][stat](val)}"><a href="{url}" target="_blank" >{val:.3e}</a></td>'))
                table.table.append(tr)
    
    page.html.body.append(table)
folder  = Path('FIBI/data/detailed_analysis/graphs/')
name = f"stats.html"
folder.mkdir(parents=True, exist_ok=True)
with open(str(folder/name),'w') as f:
    f.write(str(page.prettify()))
    
                        


### Best algorithm

In [41]:
page = bs4.BeautifulSoup("""<html><head><link rel="stylesheet" href="style4.css"></head><body></body></html>""")
for init,nsamples in product(Lassign,Lnsamples):
    page.html.body.append(bs4.BeautifulSoup(f"<h2>{init},{nsamples} samples</h2>"))
    table = bs4.BeautifulSoup(f"<table></table>")
    table.table.append(bs4.BeautifulSoup(f'<tr><th colspan="1"></th><th colspan="{len(LnPts)}">Number of points</th></tr>'))
    table.table.append(bs4.BeautifulSoup(f'<tr><th colspan="1"></th>{"".join([f"<th>{e}</th>" for e in LnPts])}</tr>'))
    for i,(nclust) in enumerate(Lnclust):
        tr = bs4.BeautifulSoup(f'<tr></tr>')
        tr.tr.append(bs4.BeautifulSoup(f'<th rowspan=1>{nclust} clusters</th>'))
        for n in LnPts:
            if n<=nclust:
                tr.tr.append(bs4.BeautifulSoup('<td></td>'))
            else:
                val = res_best[to_key(n,nsamples,nclust,init)]
                url = f"{n}_points_init_{init}_{nclust}_clusters_{nsamples}_samples.html"
                tr.tr.append(bs4.BeautifulSoup(f'<td class="{val}"><a href="{url}" target="_blank" >{val}</a></td>'))
        table.table.append(tr)
    page.html.body.append(table)

folder  = Path('FIBI/data/detailed_analysis/graphs/')
name = f"stats_best.html"
folder.mkdir(parents=True, exist_ok=True)
with open(str(folder/name),'w') as f:
    f.write(str(page.prettify()))
    

## Verification of normality

In [113]:
import json
# {'npts','nsamples','nclust', 'INIT'}
order_of_keys = ['nclust', 'npts','INIT']
keys = []
for k in res_ttest:
    d = key_to_dict(k) 
    if d['nsamples'] == str(1000):
        keys.append({**{a:d[a] for a in order_of_keys},'tags':[]})
        
keys = sorted(keys,key=lambda k:(int(k['nclust']),k['INIT'],int(k['npts'])))
tags = {"gauss":"gaussian perfectly in interval","gaussHeavy":"Gaussian with heavy tail(s)","notGauss":"not gaussian, outside of confidence interval"}
folder  = Path('FIBI/data/detailed_analysis/graphs/')
name = f"init_check_orig.json"
folder.mkdir(parents=True, exist_ok=True)
with open(str(folder/name),'w') as f:
    json.dump({"tags":tags,"init_check":keys},f,indent=4)

In [42]:
with open(str(folder/"init_check.json")) as f:
    data = json.load(f)
data_dico = {}
for e in data['init_check']:
    data_dico[to_key(e['npts'],"1000",e['nclust'],e['INIT'])] = e
map_distrib = {"g":['gaussian distribution','var(--gaussian)','gaussian'],"gh":['gaussian distribution with heavy tail(s)','var(--gaussianHeavy)','gaussian<br>with heavy<br>tail'],"ng":['not a gaussian distribution','var(--notGaussian)','not<br>gaussian']}
page = bs4.BeautifulSoup("""<html><head><link rel="stylesheet" href="style4.css"></head><body></body></html>""")
for init,nsamples in product(Lassign,Lnsamples):
    page.html.body.append(bs4.BeautifulSoup(f"<h2>{init},{nsamples} samples</h2>"))
    legend = [f"<svg width='{20}px' height='{20}px' ><rect width='{20}px' height='{20}px' fill='{e[1]}'></rect></svg>{e[0]}" for e in map_distrib.values()]
    page.html.body.append(bs4.BeautifulSoup(f"<div class='legend'><div>{''.join(legend)}</div></div>"))
    table = bs4.BeautifulSoup(f"<table></table>")
    table.table.append(bs4.BeautifulSoup(f'<tr><th colspan="1"></th><th colspan="{len(LnPts)}">Number of points</th></tr>'))
    table.table.append(bs4.BeautifulSoup(f'<tr><th colspan="1"></th>{"".join([f"<th>{e}</th>" for e in LnPts])}</tr>'))
    for i,(nclust) in enumerate(Lnclust):
        tr = bs4.BeautifulSoup(f'<tr></tr>')
        tr.tr.append(bs4.BeautifulSoup(f'<th rowspan=1>{nclust} clusters</th>'))
        for n in LnPts:
            if n<=nclust:
                tr.tr.append(bs4.BeautifulSoup('<td></td>'))
            else:
                val = data_dico[to_key(n,"1000",nclust,init)]
                if len(val) == 0:
                    raise Exception
                val = val['tags'][0]
                url = f"{n}_points_init_{init}_{nclust}_clusters_{nsamples}_samples.html#qqplot"
                tr.tr.append(bs4.BeautifulSoup(f'<td class="{val}"><a href="{url}" target="_blank" >{val}</a></td>'))
        table.table.append(tr)
    page.html.body.append(table)

folder  = Path('FIBI/data/detailed_analysis/graphs/')
name = f"stats_init_check.html"
folder.mkdir(parents=True, exist_ok=True)
with open(str(folder/name),'w') as f:
    f.write(str(page.prettify()))

## Consice view

In [58]:

page = bs4.BeautifulSoup("""<html><head><link rel="stylesheet" href="style4.css"></head><body></body></html>""")
colors_pvalue = [["Significant difference","var(--small)"],["Non significant difference","var(--big)"]]
colors_effect_size = [["Small","var(--small)"],["Medium","var(--medium)"],["Big","var(--big)"]]
for init,nsamples,[stat,colors_map] in product(Lassign,Lnsamples,[['pvalue',colors_pvalue],['effect_size',colors_effect_size]]):
    page.html.body.append(bs4.BeautifulSoup(f"<h2>{init},{nsamples} samples, {stat}</h2>"))
    legend = [f"<svg width='{20}px' height='{20}px' ><rect width='{20}px' height='{20}px' fill='{e[1]}'></rect></svg>{e[0]}" for e in colors_map]
    page.html.body.append(bs4.BeautifulSoup(f"<div class='legend'><div>{''.join(legend)}</div></div>"))
    table = bs4.BeautifulSoup(f"<table></table>")
    table.table.append(bs4.BeautifulSoup(f'<tr><th colspan="1"></th><th colspan="{len(LnPts)}">Number of points</th></tr>'))
    table.table.append(bs4.BeautifulSoup(f'<tr><th colspan="1"></th>{"".join([f"<th>{e}</th>" for e in LnPts])}</tr>'))
    for i,(nclust) in enumerate(Lnclust):
        tr = bs4.BeautifulSoup(f'<tr></tr>')
        tr.tr.append(bs4.BeautifulSoup(f'<th rowspan=1>{nclust} clusters</th>'))
        for n in LnPts:
            if n<=nclust:
                tr.tr.append(bs4.BeautifulSoup('<td></td>'))
            else:
                val = res_best[to_key(n,nsamples,nclust,init)]
                pvalue = None
                distrib = data_dico[to_key(n,"1000",nclust,init)]
                if len(val) == 0:
                    raise Exception
                distrib = distrib['tags'][0]
                if distrib in ['g','gh']:
                    test = "TTest"
                    pvalue = float(res_ttest[to_key(n,nsamples,nclust,init)][stat])
                else:
                    test = "Sign test"
                    pvalue = float(res_sign_test[to_key(n,nsamples,nclust,init)][stat])
                class_ = colors[test][stat](pvalue)
                url = f"{n}_points_init_{init}_{nclust}_clusters_{nsamples}_samples.html"
                tr.tr.append(bs4.BeautifulSoup(f'<td class="{class_}"><a href="{url}" target="_blank" title="{test}">{val}</a></td>'))
        table.table.append(tr)
    page.html.body.append(table)

folder  = Path('FIBI/data/detailed_analysis/graphs/')
name = f"stats_concise.html"
folder.mkdir(parents=True, exist_ok=True)
with open(str(folder/name),'w') as f:
    f.write(str(page.prettify()))
    

### Count number of cases pvalues, effect size

In [180]:
key_to_dict_format = lambda x: {
    'npts':int(x.split('_')[0]),
    'nsamples':int(x.split('_')[1]),
    'nclust':int(x.split('_')[2]), 
    'INIT':(x.split('_')[3])
    }
small_cohen = lambda x:abs(0.2-abs(x))<abs(0.5-abs(x))
small_zscore = lambda x:abs(0.1-abs(x))<abs(0.3-abs(x))
dico_check = {"wilcoxon":small_zscore,"ttest":small_cohen}
def cases(row,test='wilcoxon'):
    if row[test+'_pvalue'] >= 0.05 and dico_check[test](row[test+'_effect_size']):
        return 'small_diff_non_sign'
    elif row[test+'_pvalue'] < 0.05 and dico_check[test](row[test+'_effect_size']):
        return 'small_diff_sign'
    elif row[test+'_pvalue'] < 0.05 and not dico_check[test](row[test+'_effect_size']):
        return 'med_big_diff_sign'
    else:
        return 'med_big_diff_non_sign'
Lcases = ['small_diff_non_sign','small_diff_sign','med_big_diff_sign','med_big_diff_non_sign']
# Assign cases
Ldict = []
for k in res_ttest:
    wilcoxon = res_wilcoxon[k]
    ttest = res_ttest[k]
    ttest = {k:float(v) for k,v in ttest.items()}
    infos = {**key_to_dict_format(k),**{"wilcoxon_"+k:v for k,v in wilcoxon.items()},**{"ttest_"+k:v for k,v in ttest.items()}}
    infos['wilcoxon_case'] = cases(infos,test="wilcoxon")
    infos['ttest_case'] = cases(infos,test="ttest")
    if infos['npts'] < 200 or (infos['npts'] >= 200 and infos['nsamples'] <= 100):
        Ldict.append(infos)
df_stats = pd.DataFrame(Ldict)
df_stats.sort_values(by=['nclust','nsamples','INIT','npts'],inplace=True)
# Count number of cases per (nclust,nsamples)
L_stats_counts = [] 
for clust,samples in product(df_stats['nclust'].unique(),df_stats['nsamples'].unique()):
    dico_val = {'nclust':clust,'nsamples':samples}
    df_tmp1 = df_stats.query(f"nclust == {clust} & nsamples == {samples}")
    for test in ['wilcoxon','ttest']:
        L_stats2 = df_tmp1[test+'_case'].value_counts().reset_index().sort_values(by=[test+'_case'],ascending=False).to_dict(orient='records')
        stats_val = {e['index']:(e[test+'_case'],e[test+'_case']/sum(a[test+'_case'] for a in L_stats2)) for e in L_stats2}
        for v in Lcases:
            dico_cols={'case':v,'test':test,'nclust':clust,'nsamples':samples}
            if v in stats_val:
                dico_cols['count'] = stats_val[v][0]
                dico_cols['prct'] = stats_val[v][1]*100
            else:
                dico_cols['count'] = 0
                dico_cols['prct'] = 0
            L_stats_counts.append(dico_cols)
df_stats_counts = pd.DataFrame(L_stats_counts)
# Plot per (nclust,nsamples)
page = bs4.BeautifulSoup("""<html><head><link rel="stylesheet" href="style5.css"></head><body></body></html>""")
page.html.body.append(bs4.BeautifulSoup('<h1>Explanations</h1>'))
page.html.body.append(bs4.BeautifulSoup('<p>For each table of <a href="./stats.html">stats.html</a>, we split 3 cases:</p>'))
ol=bs4.BeautifulSoup('<ol></ol>')
ol.ol.append(bs4.BeautifulSoup('<li>pvalue >= 0.05, small effect_size: small non significant difference</li>'))
ol.ol.append(bs4.BeautifulSoup('<li>pvalue < 0.05, small effect_size: small significant difference</li>'))
ol.ol.append(bs4.BeautifulSoup('<li>pvalue < 0.05, medium-big effect_size: medium-big significant difference</li>'))
page.html.body.append(ol)
page.html.body.append(bs4.BeautifulSoup('<p>Each bar represent one table, and each rectangle inside each bar represent the percentage of cells in the corresponding <a href="./stats.html">stats.html</a> table that falls into one of three cases specified above (color-code associated)</p>'))
page.html.body.append(bs4.BeautifulSoup('<h1>Graphs</h1>'))

for i,(samples,test) in enumerate(product(df_stats_counts['nsamples'].unique(),df_stats_counts['test'].unique())):
    df_tmp1 = df_stats_counts.query(f"nsamples == {samples} & test == '{test}'")
    df_tmp1 = df_tmp1.rename(columns={'nclust':'Number of clusters','prct':'Proportion of cells (%)','case':'Case'})
    if i % 2 == 0:
        page.html.body.append(bs4.BeautifulSoup(f'<h2>{samples} samples</h2>'))
    html = px.bar(df_tmp1,x="Number of clusters",y="Proportion of cells (%)",color='Case',title=f"For {samples} samples, test {test}",height=500).to_html(full_html=False, include_plotlyjs='cdn')
    page.html.body.append(bs4.BeautifulSoup(html))
folder  = Path('FIBI/data/detailed_analysis/graphs/')
name = f"stats_counts.html"
folder.mkdir(parents=True, exist_ok=True)
with open(str(folder/name),'w') as f:
    f.write(str(page.prettify())) 

### pvalues variations depending of number of points

In [266]:
df_pvalues = []
df_ttest = []
for k in res_ttest:
    dico = key_to_dict(k)
    df_pvalues.append({**dico,'pvalue':res_ttest[k]['pvalue'],'effect_size':res_ttest[k]['effect_size'],'test':'ttest'})
    df_pvalues.append({**dico,'pvalue':res_wilcoxon[k]['pvalue'],'effect_size':res_wilcoxon[k]['effect_size'],'test':'wilcoxon'})
    df_ttest.append({**dico,**res_ttest[k]})
df_pvalues = pd.DataFrame(df_pvalues)
df_ttest = pd.DataFrame(df_ttest)
# df_pvalues.loc[:,'name'] = df_pvalues.loc[:,'nclust'].astype(str)+' clusters,'+df_pvalues.loc[:,'nsamples'].astype(str)+' samples,'+df_pvalues.loc[:,'INIT'].astype(str)+' init'
page = bs4.BeautifulSoup("""<html><head><link rel="stylesheet" href="style4.css"></head><body></body></html>""")
df_pvalues.loc[:,"pvalue"] = df_pvalues.loc[:,"pvalue"].astype(float)
df_pvalues.loc[:,"effect_size"] = df_pvalues.loc[:,"effect_size"].astype(float)
for nclust,init in product(df_pvalues['nclust'].unique(),df_pvalues['INIT'].unique()):
    df_visu = df_pvalues.query(f"nclust == '{nclust}' & INIT == '{init}'").copy()
    df_visu.loc[:,'name'] = df_visu.loc[:,'nsamples'].astype(str)+' samples, '+df_visu.loc[:,'test'].astype(str)
    page.html.body.append(bs4.BeautifulSoup(f"<h2 id='{nclust}_{init}'>Evolution of pvalue for {nclust} clusters, {init} init</h2>"))
    fig = px.line(df_visu,x="npts",y="pvalue",color='name',log_y=False,height=500).to_html(full_html=False, include_plotlyjs='cdn')
    page.html.body.append(bs4.BeautifulSoup(fig))
    df_visu2 = df_ttest.query(f"nclust == '{nclust}' & INIT == '{init}'").copy()
    df_visu2.loc[:,'name'] = df_visu2.loc[:,'nsamples'].astype(str)+' samples'
    fig = go.Figure()
    fig.update_layout(height=500,title="TTest evolution of mean and normalized standard deviation")
    for name in df_visu2['name'].unique():
        df_visu3 = df_visu2.query(f"name == '{name}'")
        fig.add_trace(go.Scatter(x=df_visu3['npts'],y=df_visu3['sigmaNorm'],mode='lines+markers',name='Sigma norm '+name))
        fig.add_trace(go.Scatter(x=df_visu3['npts'],y=df_visu3['mean'],mode='lines+markers',name='Mean '+name))
    fig = fig.to_html(full_html=False, include_plotlyjs='cdn')
    page.html.body.append(bs4.BeautifulSoup(fig))
folder  = Path('FIBI/data/detailed_analysis/graphs/')
name = f"pvalue_evolution.html"
folder.mkdir(parents=True, exist_ok=True)
with open(str(folder/name),'w') as f:
    f.write(str(page.prettify())) 